In [14]:
using CausalELM
using CSV
using DataFrames
using Random

In [9]:
pension_df = CSV.read("pension.csv", DataFrame)
pension_df = pension_df[:, [10, 22, 13, 14, 15, 18, 20, 17, 24, 33]]
covariates, treatment, outcome = pension_df[:, 3:end], pension_df[:, 2], pension_df[:, 1]

(9915×8 DataFrame
  Row │ age    inc    fsize  marr   twoearn  db     pira   hown  
      │ Int64  Int64  Int64  Int64  Int64    Int64  Int64  Int64 
──────┼──────────────────────────────────────────────────────────
    1 │    31  28146      5      1        0      0      0      1
    2 │    52  32634      5      0        0      0      0      1
    3 │    50  52206      3      1        1      0      1      1
    4 │    28  45252      4      1        1      0      0      0
    5 │    42  33126      3      0        0      1      0      1
    6 │    49  76860      6      1        1      1      0      1
    7 │    40  57477      4      1        1      1      0      1
    8 │    58  14637      1      0        0      0      0      0
  ⋮   │   ⋮      ⋮      ⋮      ⋮       ⋮       ⋮      ⋮      ⋮
 9909 │    28  31926      2      1        1      0      0      0
 9910 │    49  64215      4      1        1      0      1      1
 9911 │    34  13500      1      0        0      1      0      0
 9912 

In [63]:
glearner = DoubleMachineLearning(covariates, treatment, outcome)

DoubleMachineLearning([31.0 28146.0 … 0.0 1.0; 52.0 32634.0 … 0.0 1.0; … ; 41.0 56190.0 … 0.0 1.0; 28.0 26205.0 … 0.0 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-3300.0, 61010.0, 8849.0, -6013.0, -2375.0, -11000.0, -16901.0, 1000.0, 0.0, 6400.0  …  -1436.0, 4500.0, 34739.0, -750.0, 40000.0, 172.0, 836.0, 6150.0, 14499.0, -5400.0], [31.0 28146.0 … 0.0 1.0; 52.0 32634.0 … 0.0 1.0; … ; 41.0 56190.0 … 0.0 1.0; 28.0 26205.0 … 0.0 0.0], "ATE", false, "regression", true, CausalELM.relu, 8954, NaN, 5)

In [66]:
estimate_causal_effect!(glearner)

In [38]:
x, y = rand(10000, 7), rand(10000)

([0.23716522406873197 0.08463142909640708 … 0.30968748590862305 0.04725439908425155; 0.13055165056767004 0.9220378350184131 … 0.572606572207097 0.3884781806564631; … ; 0.5640916988721004 0.853346124678495 … 0.8469263452425522 0.1257190755169607; 0.6679763039334277 0.47972447662761064 … 0.37811702580338935 0.617016732528424], [0.6491269811582214, 0.5932565556655242, 0.8565916760297303, 0.7021098498625459, 0.5264840904652793, 0.7432901746261853, 0.7807974247740146, 0.540402591727013, 0.6592750061253853, 0.8705468971445318  …  0.27613447847948525, 0.23299375275857093, 0.9834654852036273, 0.26905537667480783, 0.2977201330273679, 0.2251454190526, 0.22413247851994167, 0.0759353440270586, 0.11762273465665674, 0.7904463339844465])

In [45]:
learner = CausalELM.RegularizedExtremeLearner(x, y, 32, CausalELM.relu)

Regularized Extreme Learning Machine with 32 hidden neurons

In [46]:
CausalELM.fit!(learner)

32-element Vector{Float64}:
  0.026749830711221247
  0.21033200016686496
  0.0998447220434613
 -0.0016226945603700442
  0.3597543007214425
 -0.043393923445557585
 -0.0965275383555918
  0.16851120953021403
 -0.557573006115525
 -0.2778346924700644
  ⋮
  0.5212664218550033
  0.13173644509429325
  0.5211474953702191
 -0.20661927597795182
  0.08922154206186592
  0.16653105344587832
  0.28420105086226877
  0.14469922378022404
  0.23991509930469146

In [37]:
xs = [rand(1000, 8) for i in 1:100]
ys = [rand(1000) for i in 1:100]

learners = [CausalELM.ExtremeLearner(xs[i], ys[i], 5, CausalELM.relu) for i in 1:100]
CausalELM.fit!.(learners)

100-element Vector{Vector{Float64}}:
 [0.33563059342914375, 1.554096183061739, 0.34175856928495607, 1.4843766215556682, -0.1271150310066662]
 [-0.17343604358378908, 0.12755980344503404, 0.4879726895099466, 0.4237855857253079, 0.33327314853638307]
 [0.6867049618284538, 1.7639485392494731, -0.1769622610416582, 0.8025175209234753, 0.3162124425261725]
 [0.4311107417441136, 0.3815772807360452, 0.04724625538049302, 0.35167417631976233, -0.22961157745956168]
 [0.07929165744768467, 0.42503570736716156, 0.11718878236558518, 0.6794679592330893, 0.2097825511197849]
 [0.0, -0.10187284552293427, 0.3254677777717854, 3.202266196543033e-17, 0.19784989559926286]
 [0.3612678189475889, 0.3231944876545776, -3.10093526107407e-15, 0.7815001221603154, 0.06663446895775363]
 [1.2569802097480374, -3.0084525386329504, -0.6188530616095848, 0.4304718396128743, 0.5344934682266744]
 [0.3410220874955934, 0.4997803635021601, 0.15743896412842878, 0.4836342090809235, -0.009722499096015656]
 [0.25605571278411066, 0.41395

In [39]:
mutable struct ELMEnsemble
    X::Array{Float64}
    Y::Array{Float64}
    elms::Array{CausalELM.ExtremeLearner}
end

function ELMEnsemble(
    X::Array{Float64}, 
    Y::Array{Float64}, 
    sample_size::Integer, 
    num_machines::Integer, 
    num_neurons::Integer
)
    rows = [rand(1:length(Y), length(Y)) for i in 1:num_machines]
    cols = [randperm(size(X, 2))[1:floor(Int64, sqrt(size(X, 2)))] for i ∈ 1:num_machines]
    xs, ys = [X[rows[i], cols[i]] for i ∈ eachindex(rows)], [Y[rows[i]] for i ∈ eachindex(rows)]
    elms = [CausalELM.ExtremeLearner(xs[i], ys[i], num_neurons, CausalELM.relu) for i ∈ 1:num_machines]

    return ELMEnsemble(X, Y, elms)
end

fit!(mod::ELMEnsemble) = CausalELM.fit!.(mod.elms)

fit! (generic function with 1 method)

In [50]:
ensemble = ELMEnsemble(Matrix{Float64}(covariates), Float64.(outcome[:, 1]), 10000, 100, 10)

ELMEnsemble([31.0 28146.0 … 0.0 1.0; 52.0 32634.0 … 0.0 1.0; … ; 41.0 56190.0 … 0.0 1.0; 28.0 26205.0 … 0.0 0.0], [-3300.0, 61010.0, 8849.0, -6013.0, -2375.0, -11000.0, -16901.0, 1000.0, 0.0, 6400.0  …  -1436.0, 4500.0, 34739.0, -750.0, 40000.0, 172.0, 836.0, 6150.0, 14499.0, -5400.0], CausalELM.ExtremeLearner[Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons  …  Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons, Extreme Learning Machine with 10 hidden neurons,

In [59]:
fit!(ensemble)

100-element Vector{Vector{Float64}}:
 [6211.408699229452, 1.5821332294031651, -13735.18658175283, 0.6524374453029926, 260.79520738555453, 1.4013393161668026, -1351.18915422185, 14631.142361137296, 0.9738553598743988, 1.1907620936611532]
 [-14401.515405970058, 71237.03121623177, -12585.477651933446, 14439.162597071294, -10985.595229244644, 23574.843298215033, 23123.869962055618, 23070.273691837538, 493.1701340561063, -56151.84544187152]
 [-180.9263418876608, 0.0, 2873.5351527420603, -1985.5623964348392, -2686.811852048377, 4511.355299849305, -9875.408841485112, -1349.9293238605605, 5779.2168040718025, -120.24340400725902]
 [0.0, -6257.710632510187, -19899.275681606392, -16954.679812461578, 0.0, 0.0, 22644.406308937705, 12385.177066525117, 51354.12427458451, 15260.878775158537]
 [0.0, -3.300096251119809e15, 0.0, 0.0, 1.141844324809179e15, 7.393788509724736e14, 1.7904369830116632e15, 2.6663150926029503e14, 0.0, 7.774942140694326e14]
 [-3151.838470139144, -10383.352842604001, 11084.3179493

In [13]:
m1 = GComputation(x, rand(0:1, 10000), y, regularized=false)

GComputation([0.2438970367354274 1.0203271610609299e-5 … 0.4557954201596055 0.12617408413868259; 0.9722098498565798 0.9404158702616398 … 0.572663944473092 0.4275299444804007; … ; 0.8794397256676026 0.3601868122972116 … 0.7393696907435132 0.8348951617519277; 0.014716984885172035 0.46589184307039333 … 0.7082478540550154 0.24368612561948588], [1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0], [0.203432368671021, 0.7340111063697138, 0.9246754848534284, 0.08645250409038174, 0.5651033787805703, 0.023292113627898514, 0.32903202710805357, 0.7016381615911508, 0.014335546595652393, 0.8721335250668286  …  0.7910929379901037, 0.3368161498494835, 0.40237100558857697, 0.5284804552447494, 0.7622417670440221, 0.30391987549352806, 0.9757684512845898, 0.8711831517392297, 0.3426427099660381, 0.007855605424861856], "ATE", true, "regression", false, CausalELM.relu, 8451, NaN, #undef)

In [14]:
estimate_causal_effect!(m1)

0.5764691423345073